In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/neural_network_model')
!pwd

/content/drive/.shortcut-targets-by-id/15CQlDgewua6NDmwLKRc25XqjLXZn08bL/Sanity Check for Linear CKA


In [2]:
# %tensorflow_version 1.x

from matplotlib import pyplot as plt
# from tensorflow.keras.datasets import cifar10
from keras import datasets
import tensorflow as tf
import numpy as np
import os
import time
import seaborn as sns
from itertools import permutations
from IPython.display import clear_output

from model import *

Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Settings

In [3]:
cifar_model = 10
num_classes = cifar_model
img_size = 32
n_trial = 10  # number of models with different initialization
n_sample = 20
CH0 = 16  # number of 1st layer channels
epoch_size = 26
batch_size = 128       
n_step = 50

log_path = 'logs/model/'

In [ ]:
if os.path.exists(log_path):
    raise Exception("Report path exists!")
else:
    os.makedirs(log_path)

# Load CIFAR

In [4]:
if cifar_model == 10:
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()#cifar10.load_data()
else:
    raise Exception("Invalid model!")

x_mean = np.mean(x_train)
x_std = np.std(x_train)
eps = 0.0001

train_size = len(x_train)
test_size = len(x_test)

y_train_one_hot = to_one_hot(np.squeeze(y_train), num_classes)
y_test_one_hot = to_one_hot(np.squeeze(y_test), num_classes)

# Train Networks

In [ ]:
for trial in range(n_trial):
  # Load model
  tf.reset_default_graph()
  layers, pred, X, Y, IS_TRAINING, LR, train_opt, Loss = get_model(img_size, CH0, num_classes)

  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  sess = tf.Session(config = config)
  sess.run(tf.global_variables_initializer())
  saver = tf.train.Saver()

  t0 = time.time()

  epoch = 0
  step = 0
  n_batchs = train_size // batch_size
  lr = 0.1

  while epoch < epoch_size:
      
    if epoch == 18:
        lr *= 0.1

    if epoch == 22:
        lr *= 0.1
        
    inds = np.random.permutation(train_size)
    
    for i_batch_train in range(n_batchs):
      batch_ind_train = inds[i_batch_train*batch_size: (i_batch_train+1)*batch_size]

      batch_x = (x_train[batch_ind_train].copy() - x_mean) / (x_std + eps)
      batch_x = np.concatenate((random_crop(batch_x), batch_x), axis = 0)
      batch_y = y_train_one_hot[batch_ind_train].copy()
      batch_y = np.concatenate((batch_y, batch_y), axis = 0)
              
      feed = {X: batch_x, Y: batch_y, IS_TRAINING:True, LR: lr}
      sess.run(train_opt, feed)
      step += 1

      if step % n_step == 0:
        output = []
        loss_ce = []
        for i_batch_test in range(test_size // batch_size):
          batch_x = (x_test[i_batch_test*batch_size: (i_batch_test+1)*batch_size] - x_mean) / (x_std + eps)
          batch_y = y_test_one_hot[i_batch_test*batch_size: (i_batch_test+1)*batch_size]
          output_, loss_ = sess.run([pred, Loss], feed_dict={X: batch_x, Y: batch_y, IS_TRAINING:False})
          output.extend(output_)
          loss_ce.append(loss_)
        hit = np.argmax(output,1) - np.argmax(y_test_one_hot[:len(output)],1)
        acc_test = hit[hit == 0.].shape[0] / len(hit)

        loss_test = np.mean(loss_ce)

      if step % (10*n_step) == 0:
        print('trial: %d, epoch: %.2d, loss test: %.3f, accuray test: %.3f'%(trial, epoch, loss_test, acc_test))
            
    epoch += 1

  trial_path = log_path + 'trial_' + str(trial+1) + '/'

  if os.path.exists(trial_path):
    raise Exception("Report path exists!")
  else:
    os.makedirs(trial_path) 

  print("trial: " + str(trial) + " Epoch: " + str(epoch) + ", elapsed time: %d min"% ((time.time() - t0)/60))       
  saver.save(sess, save_path=trial_path + 'trial_' + str(trial+1) + '.ckpt') 
  sess.close()

# Sampling from images

In [5]:
x_train_class =  []
for i_class in range(num_classes):
  x_train_class += [np.random.permutation(x_train[y_train[:, 0] == i_class])[:n_sample]]

x_mean = np.mean(x_train)
x_std = np.std(x_train)
eps = 0.0001

In [6]:
y_train_sampled = to_one_hot(np.array(range(num_classes)), num_classes)

In [7]:
d_list = []

In [1]:
layers

NameError: name 'layers' is not defined

# Big Loop

In [8]:
# Init d_list: 
tf.reset_default_graph()
layers, _, X, Y, IS_TRAINING, _, _, _ = get_model(img_size, CH0, num_classes)

n_layer = len(layers)
d_list = []
for i_layer in range(n_layer):
  n_w = np.prod(layers[i_layer].shape[1:])
  d_list += [np.zeros((num_classes, n_w, n_sample, n_trial))]

# Loop on models
for trial in range(n_trial):
  # Load model
  tf.reset_default_graph()
  layers, pred, X, Y, IS_TRAINING, _, _, _ = get_model(img_size, CH0, num_classes)

  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  sess = tf.Session(config = config)
  saver = tf.train.Saver()
  trial_path = trial_path = log_path + 'trial_' + str(trial+1) + '/'
  saver.restore(sess, save_path=trial_path + 'trial_' + str(trial + 1) + '.ckpt')
  # end load model

  # The accuracy of the model on the test dataset
  output = []
  for i_batch_test in range(test_size // batch_size):
      batch_x = (x_test[i_batch_test*batch_size: (i_batch_test+1)*batch_size] - x_mean) / (x_std + eps)
      batch_y = y_test_one_hot[i_batch_test*batch_size: (i_batch_test+1)*batch_size]
      output_ = sess.run(pred, feed_dict={X: batch_x, Y: batch_y, IS_TRAINING:False})
      output.extend(output_)
  hit = np.argmax(output,1) - np.argmax(y_test_one_hot[:len(output)],1)
  acc_t = hit[hit == 0.].shape[0] / len(hit)

  print('The accuracy of the model on the test dataset is: %.2f'%(100*acc_t))

  # Loop on samples of images 
  for i_sample in range(n_sample):
      x_train_sampled = np.zeros((num_classes, img_size, img_size, 3))

      for i_class in range(num_classes):
        x_train_sampled[i_class] = x_train_class[i_class][i_sample]

      x_train_sampled_norm = (x_train_sampled - x_mean)/(x_std + eps)

      # Get model's layers outputs
      u_list = sess.run(layers[:n_layer], 
                        feed_dict={X: x_train_sampled_norm, Y: y_train_sampled, IS_TRAINING:False})

      for i_layer in range(n_layer): 
        d_list[i_layer][:, :, i_sample, trial] = np.reshape(u_list[i_layer], (10, -1))
        
  sess.close()





Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.


INFO:tensorflow:Restoring parameters from logs/model/trial_1/trial_1.ckpt
The accuracy of the model on the test dataset is: 84.98
INFO:tensorflow:Restoring parameters from logs/model/trial_2/trial_2.ckpt
The accuracy of the model on the test dataset is: 84.86
INFO:tensorflow:Restoring parameters from logs/model/trial_3/trial_3.ckpt
The accuracy of the model on the test dataset is: 85.47
INFO:tensorflow:Restoring parameters from logs/model/trial_4/trial_4.ckpt
The accuracy of the model on the test dataset is: 85.46
INFO:tensorflow:Restoring parameters from logs/model/trial_5/trial_5.ckpt
The accuracy of the model on the test dataset is: 84.96
INFO:tensorflow:Restoring parameters from logs/model/trial_6/trial_6.ckpt
The accuracy of the model on the test dataset is: 85.05
INFO:tensorflow:Rest

In [9]:
import scipy.io as sio
sio.savemat('d_list.mat', {'l1':d_list[0],'l2':d_list[1],'l3':d_list[2],'l4':d_list[3],'l5':d_list[4]
                          ,'l6':d_list[5],'l7':d_list[6],'l8':d_list[7],'l9':d_list[8]})


(10, 64, 20, 10)